In [96]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy import sparse 
import time


from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import os
from sklearn.feature_selection import chi2, SelectPercentile,SelectKBest
import lightgbm as lgb

In [97]:
train = pd.read_pickle(FILE.train_final.value)
test = pd.read_pickle(FILE.test_final.value)
train.drop('time_hour',axis = 1,inplace=True)
test.drop('time_hour',axis = 1,inplace=True)
data = pd.concat([train, test], axis=0, ignore_index=True,sort=False)


In [98]:
data = data.fillna(-1)
os.environ['TZ'] = 'Asia/Shanghai'
data['day'] = data['time'].apply(lambda x: int(time.strftime("%d", time.localtime(x))))
data['hour'] = data['time'].apply(lambda x: int(time.strftime("%H", time.localtime(x))))
data['label'] = data.click.astype(int)
del data['click']

In [99]:
bool_feature = ['creative_is_jump', 'creative_is_download', 'creative_is_js', 'creative_is_voicead',
                'creative_has_deeplink', 'app_paid']
for i in bool_feature:
    data[i] = data[i].astype(int)

data['advert_industry_inner_1'] = data['advert_industry_inner'].apply(lambda x: x.split('_')[0])

ad_cate_feature = ['adid', 'advert_id', 'orderid', 'advert_industry_inner_1', 'advert_industry_inner', 'advert_name',
                   'campaign_id', 'creative_id', 'creative_type', 'creative_tp_dnf', 'creative_has_deeplink',
                   'creative_is_jump', 'creative_is_download']

media_cate_feature = ['app_cate_id', 'f_channel', 'app_id', 'inner_slot_id']

content_cate_feature = ['city', 'carrier', 'province', 'nnt', 'devtype', 'osv', 'os', 'make', 'model']

origin_cate_list = ad_cate_feature + media_cate_feature + content_cate_feature

for i in origin_cate_list:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(0, data[i].nunique()))))
    
cate_feature = origin_cate_list

num_feature = ['creative_width', 'creative_height', 'hour']

feature = cate_feature + num_feature
print(len(feature), feature)

29 ['adid', 'advert_id', 'orderid', 'advert_industry_inner_1', 'advert_industry_inner', 'advert_name', 'campaign_id', 'creative_id', 'creative_type', 'creative_tp_dnf', 'creative_has_deeplink', 'creative_is_jump', 'creative_is_download', 'app_cate_id', 'f_channel', 'app_id', 'inner_slot_id', 'city', 'carrier', 'province', 'nnt', 'devtype', 'osv', 'os', 'make', 'model', 'creative_width', 'creative_height', 'hour']


In [100]:
predict = data[data.label == -1].copy()
predict_result = predict[['instance_id']].copy()
predict_result['predicted_score'] = 0
predict_x = predict.drop('label', axis=1)

train_x = data[data.label != -1].copy()
train_y = data[data.label != -1].label.values

# prepare one-hot

In [101]:
special = ['model','make']
try:
    raise FileNotFoundError
    base_train_csr = pickle.load(open(FILE.one_hot_base_train.value,'rb'))
    base_predict_csr = pickle.load(open(FILE.one_hot_base_test.value,'rb'))
    print('historical base was loaded!')
except Exception:
    print('not found one_hot from history! generate from scratch!')
    base_train_csr = sparse.csr_matrix((len(train), 0))
    base_predict_csr = sparse.csr_matrix((len(test), 0))
    
    enc = OneHotEncoder()
    for feature in tqdm(cate_feature):
        try:
            onehot_train = pickle.load(open(FILE.one_hot_train_formatter.value.format(feature),'rb'))
            onehot_test = pickle.load(open(FILE.one_hot_test_formatter.value.format(feature),'rb'))
            if feature in special:
                pass
#                 raise FileNotFoundError
        except FileNotFoundError:
            print(feature)
                
            enc.fit(data[feature].values.reshape(-1, 1))
            onehot_train = enc.transform(train_x[feature].values.reshape(-1, 1))
            print(onehot_train.shape)
            onehot_test = enc.transform(predict_x[feature].values.reshape(-1, 1))
            pickle.dump(onehot_train,open(FILE.one_hot_train_formatter.value.format(feature),'wb'))
            pickle.dump(onehot_test,open(FILE.one_hot_test_formatter.value.format(feature),'wb'))


        base_train_csr = sparse.hstack((base_train_csr, onehot_train), 'csr',
                                       'bool')
        base_predict_csr = sparse.hstack((base_predict_csr, onehot_test),
                                         'csr',
                                         'bool')
    print('one-hot prepared !')

    cv = CountVectorizer(min_df=1)
    
    for feature in ['user_tags']:
        try:
            onehot_train = pickle.load(open(FILE.one_hot_train_formatter.value.format(feature),'rb'))
            onehot_test = pickle.load(open(FILE.one_hot_test_formatter.value.format(feature),'rb'))
#             raise FileNotFoundError
        except FileNotFoundError:
            
            data[feature] = data[feature].astype(str)
            cv.fit(data[feature])
            onehot_train = cv.transform(train_x[feature].astype(str))
            onehot_test = cv.transform(predict_x[feature].astype(str))
            pickle.dump(onehot_train,open(FILE.one_hot_train_formatter.value.format(feature),'wb'))
            pickle.dump(onehot_test,open(FILE.one_hot_test_formatter.value.format(feature),'wb'))
    base_train_csr = sparse.hstack((base_train_csr, onehot_train), 'csr', 'bool')
    base_predict_csr = sparse.hstack((base_predict_csr, onehot_test), 'csr',
                             'bool')
    pickle.dump(base_train_csr,open(FILE.one_hot_base_train.value,'wb'))
    pickle.dump(base_predict_csr,open(FILE.one_hot_base_test.value,'wb'))

    
print(base_train_csr.shape)
print(base_predict_csr.shape)


  0%|          | 0/26 [00:00<?, ?it/s]

not found one_hot from history! generate from scratch!


100%|██████████| 26/26 [00:26<00:00,  1.02s/it]


one-hot prepared !
(2992639, 33966)
(80276, 33966)


In [102]:
train_csr = sparse.hstack(
    (sparse.csr_matrix(train_x[num_feature]), base_train_csr), 'csr').astype(
    'float32')
predict_csr = sparse.hstack(
    (sparse.csr_matrix(predict_x[num_feature]), base_predict_csr), 'csr').astype('float32')
print(train_csr.shape)
# feature_select = SelectPercentile(chi2, percentile=99)
feature_select = SelectKBest(chi2,k=25000)
feature_select.fit(train_csr, train_y)
train_csr = feature_select.transform(train_csr)
predict_csr = feature_select.transform(predict_csr)
print('feature select')
print(train_csr.shape)

(2992639, 33969)
feature select
(2992639, 25000)


In [109]:
from scipy.sparse import csr_matrix
def save_csr(matrix,file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    np.save(data_file,matrix.data)
    print('data save')
    np.save(indices_file,matrix.indices)
    print('indices save')
    np.save(indptr_file,matrix.indptr)
    print('indptr save')
    print('save done!')
    
def load_csr(file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    data = np.load(data_file)
    print('data load')
    indices = np.load(indices_file)
    print('indices load')
    indptr = np.load(indptr_file)
    print('indptr load')
    csr = csr_matrix((data, indices, indptr))
    return csr

In [107]:
save_csr(predict_csr)

save done!


In [110]:
csr = load_csr(file_prefix='csr')

# append feature

In [94]:
roun=2
append_count = pd.read_pickle(FILE.X_fe_agg_count_formater.value.format(roun)).reset_index(drop=True)
print(append_count.shape)
append_timeNext = pd.read_pickle(FILE.X_fe_agg_time_next_formater.value.format(roun)).reset_index(drop=True)
print(append_timeNext.shape)
append_timeCount = pd.read_pickle(FILE.X_fe_agg_time_count_formater.value.format(roun)).reset_index(drop=True)
print(append_timeCount.shape)
append_shiyi = pd.read_pickle('../../data/features/agg/cnt41_col192.pkl').reset_index(drop=True)
print(append_shiyi.shape)
append_ignore = []
append_ignore2 = []
for col in append_count.columns:
    if 'cumulative' in col :
        if col == 'instance_id':
            continue
        append_ignore.append(col)
append_ignore.append('instance_id')
append_count = append_count[append_ignore].copy()

# for col in append_timeCount.columns:
#     if 'next' in col :
#         append_ignore2.append(col)
# append_ignore2.append('instance_id')
# append_timeCount = append_timeCount[append_ignore2].copy()
        

# fe_list = [append_count,append_timeCount,append_timeNext]
fe_list = [append_count,append_timeCount,append_timeNext]

def fe_appends(train_df,test_df,fe_list=[],on_col='instance_id',cut_bin=80):
    train = train_df.copy()
    test = test_df.copy()
    for X_fe in fe_list:
        if cut_bin is not None:
            print('cut into bins')
            for col in tqdm(X_fe.columns):
                if col ==on_col:
                    continue
                X_fe[col] = pd.cut(X_fe[col], bins=cut_bin, labels=False, retbins=True, right=False)[0]  
        train_order = train[[on_col]].copy()
        train_order['ori_order'] = train_order.index
        test_order = test[[on_col]].copy()
        test_order['ori_order'] = test_order.index

        X_fe_order = X_fe[[on_col]].copy()
        X_fe_order['fe_order'] = X_fe_order.index
        X_fe_train = X_fe_order.merge(train_order,on=on_col,how='inner')
        X_fe_train = X_fe_train.sort_values('ori_order',ascending=True)
        X_fe_test = X_fe_order.merge(test_order,on=on_col,how='inner')
        X_fe_test = X_fe_test.sort_values('ori_order',ascending=True)
        index_order_train = X_fe_train.fe_order.values
        index_order_test = X_fe_test.fe_order.values

        X_fe_train = X_fe.iloc[index_order_train].copy()
        print(X_fe_train.shape)

        X_fe_test = X_fe.iloc[index_order_test].copy()
        print(X_fe_test.shape)

        assert np.sum(X_fe_train.instance_id.values != train.instance_id.values) == 0
        assert np.sum(X_fe_test.instance_id.values != test.instance_id.values) == 0

        for col in tqdm(X_fe_train.columns):
            if col == on_col:
                continue
            else:
                train[col] = X_fe_train[col].values
                test[col] = X_fe_test[col].values
        print(train.shape)
        print(test.shape)
        return train,test
train_appends,test_appends = fe_appends(train[['instance_id']],test[['instance_id']],fe_list = [append_shiyi])
train_appends.drop('instance_id',axis=1,inplace=True)
test_appends.drop('instance_id',axis=1,inplace=True)
train_csr_f = sparse.hstack(
    (sparse.csr_matrix(train_appends), train_csr), 'csr').astype(
    'float32')
predict_csr_f = sparse.hstack(
    (sparse.csr_matrix(test_appends), predict_csr), 'csr').astype('float32')
print(train_csr_f.shape)




# train_appends,test_appends = fe_appends(train[['instance_id']],test[['instance_id']],fe_list = [append_count],cut_bin=80)
# train_appends.drop('instance_id',axis=1,inplace=True)
# test_appends.drop('instance_id',axis=1,inplace=True)
# train_csr_f = sparse.hstack(
#     (sparse.csr_matrix(train_appends), train_csr_f), 'csr').astype(
#     'float32')
# predict_csr_f = sparse.hstack(
#     (sparse.csr_matrix(test_appends), predict_csr_f), 'csr').astype('float32')
# print(train_csr_f.shape)

(3072915, 1135)
(3072915, 757)
(3072915, 2269)
(3072915, 193)


  0%|          | 0/193 [00:00<?, ?it/s]

cut into bins


  8%|▊         | 15/193 [00:00<00:01, 146.51it/s]

(2992639, 193)
(80276, 193)


100%|██████████| 193/193 [00:02<00:00, 67.33it/s]


(2992639, 193)
(80276, 193)
(2992639, 25192)


# Define LGB

In [53]:
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=32, reg_alpha=0, reg_lambda=0.1,
    max_depth=-1, n_estimators=5000, objective='binary',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, random_state=1001, n_jobs=16
)


In [93]:
skf = StratifiedKFold(n_splits=5, random_state=1001, shuffle=True)
best_score = []
for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y)):
    lgb_model.fit(train_csr_f[train_index], train_y[train_index],
                  eval_set=[(train_csr_f[train_index], train_y[train_index]),
                            (train_csr_f[test_index], train_y[test_index])], early_stopping_rounds=100,verbose=10)
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    print('cv:',np.mean(best_score))
    test_pred = lgb_model.predict_proba(predict_csr_f, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred

Training until validation scores don't improve for 100 rounds.
[10]	valid_0's binary_logloss: 0.457193	valid_1's binary_logloss: 0.457284
[20]	valid_0's binary_logloss: 0.438768	valid_1's binary_logloss: 0.438949
[30]	valid_0's binary_logloss: 0.429786	valid_1's binary_logloss: 0.430042
[40]	valid_0's binary_logloss: 0.425074	valid_1's binary_logloss: 0.425371
[50]	valid_0's binary_logloss: 0.42245	valid_1's binary_logloss: 0.422798
[60]	valid_0's binary_logloss: 0.420994	valid_1's binary_logloss: 0.421381
[70]	valid_0's binary_logloss: 0.42004	valid_1's binary_logloss: 0.420462
[80]	valid_0's binary_logloss: 0.419352	valid_1's binary_logloss: 0.419813
[90]	valid_0's binary_logloss: 0.41885	valid_1's binary_logloss: 0.419349
[100]	valid_0's binary_logloss: 0.418434	valid_1's binary_logloss: 0.418973
[110]	valid_0's binary_logloss: 0.418067	valid_1's binary_logloss: 0.418648
[120]	valid_0's binary_logloss: 0.417757	valid_1's binary_logloss: 0.418376
[130]	valid_0's binary_logloss: 0.417

[1090]	valid_0's binary_logloss: 0.408187	valid_1's binary_logloss: 0.41528
[1100]	valid_0's binary_logloss: 0.408118	valid_1's binary_logloss: 0.415274
[1110]	valid_0's binary_logloss: 0.408045	valid_1's binary_logloss: 0.415274
[1120]	valid_0's binary_logloss: 0.407973	valid_1's binary_logloss: 0.415266
[1130]	valid_0's binary_logloss: 0.407889	valid_1's binary_logloss: 0.415261
[1140]	valid_0's binary_logloss: 0.407818	valid_1's binary_logloss: 0.415258
[1150]	valid_0's binary_logloss: 0.407747	valid_1's binary_logloss: 0.415255
[1160]	valid_0's binary_logloss: 0.407672	valid_1's binary_logloss: 0.415247
[1170]	valid_0's binary_logloss: 0.407602	valid_1's binary_logloss: 0.41525
[1180]	valid_0's binary_logloss: 0.40753	valid_1's binary_logloss: 0.415254
[1190]	valid_0's binary_logloss: 0.40746	valid_1's binary_logloss: 0.415251
[1200]	valid_0's binary_logloss: 0.407391	valid_1's binary_logloss: 0.415245
[1210]	valid_0's binary_logloss: 0.407317	valid_1's binary_logloss: 0.415245
[12

KeyboardInterrupt: 

# shiyi 196
[0.41484805597793023, 0.41454227434998325, 0.41446901736707475, 0.4144595048790526, 0.41492599981274836]
cv: 0.4146489704773579

In [83]:
skf = StratifiedKFold(n_splits=5, random_state=1001, shuffle=True)
best_score = []
for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y)):
    lgb_model.fit(train_csr_f[train_index], train_y[train_index],
                  eval_set=[(train_csr_f[train_index], train_y[train_index]),
                            (train_csr_f[test_index], train_y[test_index])], early_stopping_rounds=100,verbose=10)
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    print('cv:',np.mean(best_score))
    test_pred = lgb_model.predict_proba(predict_csr_f, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred

Training until validation scores don't improve for 100 rounds.
[10]	valid_0's binary_logloss: 0.457165	valid_1's binary_logloss: 0.457269
[20]	valid_0's binary_logloss: 0.438837	valid_1's binary_logloss: 0.439016
[30]	valid_0's binary_logloss: 0.429859	valid_1's binary_logloss: 0.430095
[40]	valid_0's binary_logloss: 0.425145	valid_1's binary_logloss: 0.425437
[50]	valid_0's binary_logloss: 0.422547	valid_1's binary_logloss: 0.42287
[60]	valid_0's binary_logloss: 0.421046	valid_1's binary_logloss: 0.421402
[70]	valid_0's binary_logloss: 0.42012	valid_1's binary_logloss: 0.420524
[80]	valid_0's binary_logloss: 0.419463	valid_1's binary_logloss: 0.419914
[90]	valid_0's binary_logloss: 0.418932	valid_1's binary_logloss: 0.419416
[100]	valid_0's binary_logloss: 0.418526	valid_1's binary_logloss: 0.419057
[110]	valid_0's binary_logloss: 0.418159	valid_1's binary_logloss: 0.418727
[120]	valid_0's binary_logloss: 0.417819	valid_1's binary_logloss: 0.418417
[130]	valid_0's binary_logloss: 0.41

[1090]	valid_0's binary_logloss: 0.409055	valid_1's binary_logloss: 0.415039
[1100]	valid_0's binary_logloss: 0.408995	valid_1's binary_logloss: 0.415037
[1110]	valid_0's binary_logloss: 0.408934	valid_1's binary_logloss: 0.415037
[1120]	valid_0's binary_logloss: 0.408873	valid_1's binary_logloss: 0.415032
[1130]	valid_0's binary_logloss: 0.40881	valid_1's binary_logloss: 0.415028
[1140]	valid_0's binary_logloss: 0.408751	valid_1's binary_logloss: 0.415022
[1150]	valid_0's binary_logloss: 0.40869	valid_1's binary_logloss: 0.41502
[1160]	valid_0's binary_logloss: 0.408626	valid_1's binary_logloss: 0.41502
[1170]	valid_0's binary_logloss: 0.408568	valid_1's binary_logloss: 0.415019
[1180]	valid_0's binary_logloss: 0.408504	valid_1's binary_logloss: 0.415016
[1190]	valid_0's binary_logloss: 0.408438	valid_1's binary_logloss: 0.415008
[1200]	valid_0's binary_logloss: 0.408375	valid_1's binary_logloss: 0.415006
[1210]	valid_0's binary_logloss: 0.408313	valid_1's binary_logloss: 0.415007
[12

[180]	valid_0's binary_logloss: 0.416556	valid_1's binary_logloss: 0.416854
[190]	valid_0's binary_logloss: 0.416392	valid_1's binary_logloss: 0.416738
[200]	valid_0's binary_logloss: 0.416236	valid_1's binary_logloss: 0.416629
[210]	valid_0's binary_logloss: 0.416087	valid_1's binary_logloss: 0.416523
[220]	valid_0's binary_logloss: 0.415941	valid_1's binary_logloss: 0.416422
[230]	valid_0's binary_logloss: 0.415804	valid_1's binary_logloss: 0.41633
[240]	valid_0's binary_logloss: 0.415662	valid_1's binary_logloss: 0.41624
[250]	valid_0's binary_logloss: 0.415528	valid_1's binary_logloss: 0.416161
[260]	valid_0's binary_logloss: 0.415404	valid_1's binary_logloss: 0.416097
[270]	valid_0's binary_logloss: 0.4153	valid_1's binary_logloss: 0.416046
[280]	valid_0's binary_logloss: 0.415189	valid_1's binary_logloss: 0.416001
[290]	valid_0's binary_logloss: 0.415077	valid_1's binary_logloss: 0.415933
[300]	valid_0's binary_logloss: 0.414963	valid_1's binary_logloss: 0.415871
[310]	valid_0's 

[1260]	valid_0's binary_logloss: 0.408126	valid_1's binary_logloss: 0.414639
[1270]	valid_0's binary_logloss: 0.408066	valid_1's binary_logloss: 0.414638
[1280]	valid_0's binary_logloss: 0.408005	valid_1's binary_logloss: 0.414637
[1290]	valid_0's binary_logloss: 0.407945	valid_1's binary_logloss: 0.414635
[1300]	valid_0's binary_logloss: 0.407883	valid_1's binary_logloss: 0.414639
[1310]	valid_0's binary_logloss: 0.407822	valid_1's binary_logloss: 0.414637
[1320]	valid_0's binary_logloss: 0.407762	valid_1's binary_logloss: 0.414631
[1330]	valid_0's binary_logloss: 0.407699	valid_1's binary_logloss: 0.414625
[1340]	valid_0's binary_logloss: 0.40764	valid_1's binary_logloss: 0.414623
[1350]	valid_0's binary_logloss: 0.407579	valid_1's binary_logloss: 0.414616
[1360]	valid_0's binary_logloss: 0.407521	valid_1's binary_logloss: 0.414616
[1370]	valid_0's binary_logloss: 0.407464	valid_1's binary_logloss: 0.414614
[1380]	valid_0's binary_logloss: 0.40741	valid_1's binary_logloss: 0.414616
[

[90]	valid_0's binary_logloss: 0.418948	valid_1's binary_logloss: 0.419087
[100]	valid_0's binary_logloss: 0.418556	valid_1's binary_logloss: 0.418715
[110]	valid_0's binary_logloss: 0.418191	valid_1's binary_logloss: 0.418387
[120]	valid_0's binary_logloss: 0.417855	valid_1's binary_logloss: 0.418086
[130]	valid_0's binary_logloss: 0.417597	valid_1's binary_logloss: 0.417855
[140]	valid_0's binary_logloss: 0.417348	valid_1's binary_logloss: 0.417632
[150]	valid_0's binary_logloss: 0.417107	valid_1's binary_logloss: 0.417423
[160]	valid_0's binary_logloss: 0.416913	valid_1's binary_logloss: 0.417264
[170]	valid_0's binary_logloss: 0.416718	valid_1's binary_logloss: 0.417099
[180]	valid_0's binary_logloss: 0.416544	valid_1's binary_logloss: 0.416963
[190]	valid_0's binary_logloss: 0.41637	valid_1's binary_logloss: 0.416845
[200]	valid_0's binary_logloss: 0.416212	valid_1's binary_logloss: 0.416726
[210]	valid_0's binary_logloss: 0.416068	valid_1's binary_logloss: 0.416632
[220]	valid_0'

...

[100]	valid_0's binary_logloss: 0.418495	valid_1's binary_logloss: 0.418705
[110]	valid_0's binary_logloss: 0.418147	valid_1's binary_logloss: 0.418405
[120]	valid_0's binary_logloss: 0.417839	valid_1's binary_logloss: 0.418133
[130]	valid_0's binary_logloss: 0.417544	valid_1's binary_logloss: 0.417879
[140]	valid_0's binary_logloss: 0.417299	valid_1's binary_logloss: 0.417673
[150]	valid_0's binary_logloss: 0.417076	valid_1's binary_logloss: 0.417486
[160]	valid_0's binary_logloss: 0.41687	valid_1's binary_logloss: 0.417318
[170]	valid_0's binary_logloss: 0.416671	valid_1's binary_logloss: 0.417157
[180]	valid_0's binary_logloss: 0.416492	valid_1's binary_logloss: 0.417022
[190]	valid_0's binary_logloss: 0.416323	valid_1's binary_logloss: 0.416899
[200]	valid_0's binary_logloss: 0.416159	valid_1's binary_logloss: 0.416767
[210]	valid_0's binary_logloss: 0.416003	valid_1's binary_logloss: 0.416652
[220]	valid_0's binary_logloss: 0.41587	valid_1's binary_logloss: 0.416567
[230]	valid_0'

[1180]	valid_0's binary_logloss: 0.408594	valid_1's binary_logloss: 0.414726
[1190]	valid_0's binary_logloss: 0.408532	valid_1's binary_logloss: 0.414728
[1200]	valid_0's binary_logloss: 0.408474	valid_1's binary_logloss: 0.414725
[1210]	valid_0's binary_logloss: 0.408411	valid_1's binary_logloss: 0.414714
[1220]	valid_0's binary_logloss: 0.408346	valid_1's binary_logloss: 0.414707
[1230]	valid_0's binary_logloss: 0.408287	valid_1's binary_logloss: 0.4147
[1240]	valid_0's binary_logloss: 0.40823	valid_1's binary_logloss: 0.4147
[1250]	valid_0's binary_logloss: 0.408166	valid_1's binary_logloss: 0.414694
[1260]	valid_0's binary_logloss: 0.408105	valid_1's binary_logloss: 0.414691
[1270]	valid_0's binary_logloss: 0.408045	valid_1's binary_logloss: 0.414689
[1280]	valid_0's binary_logloss: 0.407986	valid_1's binary_logloss: 0.414679
[1290]	valid_0's binary_logloss: 0.407926	valid_1's binary_logloss: 0.414666
[1300]	valid_0's binary_logloss: 0.407867	valid_1's binary_logloss: 0.414662
[131

[2250]	valid_0's binary_logloss: 0.402645	valid_1's binary_logloss: 0.414496
[2260]	valid_0's binary_logloss: 0.402596	valid_1's binary_logloss: 0.414494
[2270]	valid_0's binary_logloss: 0.402543	valid_1's binary_logloss: 0.414493
[2280]	valid_0's binary_logloss: 0.40249	valid_1's binary_logloss: 0.414493
[2290]	valid_0's binary_logloss: 0.402436	valid_1's binary_logloss: 0.414489
[2300]	valid_0's binary_logloss: 0.402376	valid_1's binary_logloss: 0.414488
[2310]	valid_0's binary_logloss: 0.40232	valid_1's binary_logloss: 0.414483
[2320]	valid_0's binary_logloss: 0.402271	valid_1's binary_logloss: 0.414483
[2330]	valid_0's binary_logloss: 0.402221	valid_1's binary_logloss: 0.414484
[2340]	valid_0's binary_logloss: 0.402166	valid_1's binary_logloss: 0.414481
[2350]	valid_0's binary_logloss: 0.402115	valid_1's binary_logloss: 0.414477
[2360]	valid_0's binary_logloss: 0.402061	valid_1's binary_logloss: 0.414474
[2370]	valid_0's binary_logloss: 0.402013	valid_1's binary_logloss: 0.414475
[

[670]	valid_0's binary_logloss: 0.411816	valid_1's binary_logloss: 0.415407
[680]	valid_0's binary_logloss: 0.411741	valid_1's binary_logloss: 0.415392
[690]	valid_0's binary_logloss: 0.411674	valid_1's binary_logloss: 0.415384
[700]	valid_0's binary_logloss: 0.411604	valid_1's binary_logloss: 0.415374
[710]	valid_0's binary_logloss: 0.411533	valid_1's binary_logloss: 0.415365
[720]	valid_0's binary_logloss: 0.411465	valid_1's binary_logloss: 0.415357
[730]	valid_0's binary_logloss: 0.41139	valid_1's binary_logloss: 0.415344
[740]	valid_0's binary_logloss: 0.411326	valid_1's binary_logloss: 0.415333
[750]	valid_0's binary_logloss: 0.411257	valid_1's binary_logloss: 0.415326
[760]	valid_0's binary_logloss: 0.411186	valid_1's binary_logloss: 0.415309
[770]	valid_0's binary_logloss: 0.41112	valid_1's binary_logloss: 0.415296
[780]	valid_0's binary_logloss: 0.411054	valid_1's binary_logloss: 0.415292
[790]	valid_0's binary_logloss: 0.410988	valid_1's binary_logloss: 0.415285
[800]	valid_0'

[1750]	valid_0's binary_logloss: 0.405228	valid_1's binary_logloss: 0.414963
[1760]	valid_0's binary_logloss: 0.405176	valid_1's binary_logloss: 0.41496
[1770]	valid_0's binary_logloss: 0.405123	valid_1's binary_logloss: 0.414961
[1780]	valid_0's binary_logloss: 0.405069	valid_1's binary_logloss: 0.41496
[1790]	valid_0's binary_logloss: 0.405011	valid_1's binary_logloss: 0.414953
[1800]	valid_0's binary_logloss: 0.404958	valid_1's binary_logloss: 0.414952
[1810]	valid_0's binary_logloss: 0.404901	valid_1's binary_logloss: 0.414951
[1820]	valid_0's binary_logloss: 0.404847	valid_1's binary_logloss: 0.414954
[1830]	valid_0's binary_logloss: 0.404793	valid_1's binary_logloss: 0.414949
[1840]	valid_0's binary_logloss: 0.404737	valid_1's binary_logloss: 0.414945
[1850]	valid_0's binary_logloss: 0.40468	valid_1's binary_logloss: 0.414948
[1860]	valid_0's binary_logloss: 0.40463	valid_1's binary_logloss: 0.41495
[1870]	valid_0's binary_logloss: 0.404576	valid_1's binary_logloss: 0.41495
[1880

# reverse cumulative count! just reverse -- 0.4149
[0.4148858898168986, 0.4146883252626958, 0.41459811479927555, 0.4147769010588883, 0.41512814995148156]
cv: 0.4148154761778479
test mean: 0.19420425145654233

In [60]:
skf = StratifiedKFold(n_splits=5, random_state=1001, shuffle=True)
best_score = []
for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y)):
    lgb_model.fit(train_csr_f[train_index], train_y[train_index],
                  eval_set=[(train_csr_f[train_index], train_y[train_index]),
                            (train_csr_f[test_index], train_y[test_index])], early_stopping_rounds=100,verbose=10)
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    print('cv:',np.mean(best_score))
    test_pred = lgb_model.predict_proba(predict_csr_f, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred

Training until validation scores don't improve for 100 rounds.
[10]	valid_0's binary_logloss: 0.458378	valid_1's binary_logloss: 0.458449
[20]	valid_0's binary_logloss: 0.440296	valid_1's binary_logloss: 0.440445
[30]	valid_0's binary_logloss: 0.431387	valid_1's binary_logloss: 0.431589
[40]	valid_0's binary_logloss: 0.426716	valid_1's binary_logloss: 0.42698
[50]	valid_0's binary_logloss: 0.424065	valid_1's binary_logloss: 0.424366
[60]	valid_0's binary_logloss: 0.422486	valid_1's binary_logloss: 0.422825
[70]	valid_0's binary_logloss: 0.42139	valid_1's binary_logloss: 0.421765
[80]	valid_0's binary_logloss: 0.420652	valid_1's binary_logloss: 0.421055
[90]	valid_0's binary_logloss: 0.420064	valid_1's binary_logloss: 0.420496
[100]	valid_0's binary_logloss: 0.419572	valid_1's binary_logloss: 0.420039
[110]	valid_0's binary_logloss: 0.419162	valid_1's binary_logloss: 0.41966
[120]	valid_0's binary_logloss: 0.418798	valid_1's binary_logloss: 0.419317
[130]	valid_0's binary_logloss: 0.418

[1090]	valid_0's binary_logloss: 0.409869	valid_1's binary_logloss: 0.415214
[1100]	valid_0's binary_logloss: 0.409808	valid_1's binary_logloss: 0.415204
[1110]	valid_0's binary_logloss: 0.409748	valid_1's binary_logloss: 0.415196
[1120]	valid_0's binary_logloss: 0.409693	valid_1's binary_logloss: 0.415193
[1130]	valid_0's binary_logloss: 0.409635	valid_1's binary_logloss: 0.415193
[1140]	valid_0's binary_logloss: 0.409581	valid_1's binary_logloss: 0.41519
[1150]	valid_0's binary_logloss: 0.409526	valid_1's binary_logloss: 0.415183
[1160]	valid_0's binary_logloss: 0.409466	valid_1's binary_logloss: 0.415181
[1170]	valid_0's binary_logloss: 0.409412	valid_1's binary_logloss: 0.415178
[1180]	valid_0's binary_logloss: 0.409353	valid_1's binary_logloss: 0.415173
[1190]	valid_0's binary_logloss: 0.409298	valid_1's binary_logloss: 0.415172
[1200]	valid_0's binary_logloss: 0.40924	valid_1's binary_logloss: 0.415168
[1210]	valid_0's binary_logloss: 0.409184	valid_1's binary_logloss: 0.41517
[1

[2160]	valid_0's binary_logloss: 0.404156	valid_1's binary_logloss: 0.414901
[2170]	valid_0's binary_logloss: 0.404105	valid_1's binary_logloss: 0.4149
[2180]	valid_0's binary_logloss: 0.404055	valid_1's binary_logloss: 0.414902
[2190]	valid_0's binary_logloss: 0.404006	valid_1's binary_logloss: 0.4149
[2200]	valid_0's binary_logloss: 0.403959	valid_1's binary_logloss: 0.414899
[2210]	valid_0's binary_logloss: 0.403911	valid_1's binary_logloss: 0.414896
[2220]	valid_0's binary_logloss: 0.403862	valid_1's binary_logloss: 0.414898
[2230]	valid_0's binary_logloss: 0.403811	valid_1's binary_logloss: 0.414895
[2240]	valid_0's binary_logloss: 0.403763	valid_1's binary_logloss: 0.414896
[2250]	valid_0's binary_logloss: 0.403711	valid_1's binary_logloss: 0.414898
[2260]	valid_0's binary_logloss: 0.403659	valid_1's binary_logloss: 0.414897
[2270]	valid_0's binary_logloss: 0.403609	valid_1's binary_logloss: 0.414892
[2280]	valid_0's binary_logloss: 0.403556	valid_1's binary_logloss: 0.414888
[22

[840]	valid_0's binary_logloss: 0.411529	valid_1's binary_logloss: 0.415216
[850]	valid_0's binary_logloss: 0.411466	valid_1's binary_logloss: 0.415203
[860]	valid_0's binary_logloss: 0.411404	valid_1's binary_logloss: 0.4152
[870]	valid_0's binary_logloss: 0.411341	valid_1's binary_logloss: 0.415191
[880]	valid_0's binary_logloss: 0.411281	valid_1's binary_logloss: 0.415186
[890]	valid_0's binary_logloss: 0.41122	valid_1's binary_logloss: 0.415177
[900]	valid_0's binary_logloss: 0.41116	valid_1's binary_logloss: 0.415167
[910]	valid_0's binary_logloss: 0.411098	valid_1's binary_logloss: 0.415157
[920]	valid_0's binary_logloss: 0.411032	valid_1's binary_logloss: 0.415151
[930]	valid_0's binary_logloss: 0.410971	valid_1's binary_logloss: 0.415143
[940]	valid_0's binary_logloss: 0.41091	valid_1's binary_logloss: 0.415133
[950]	valid_0's binary_logloss: 0.410852	valid_1's binary_logloss: 0.415127
[960]	valid_0's binary_logloss: 0.41079	valid_1's binary_logloss: 0.415114
[970]	valid_0's bi

[1910]	valid_0's binary_logloss: 0.405498	valid_1's binary_logloss: 0.414753
[1920]	valid_0's binary_logloss: 0.405443	valid_1's binary_logloss: 0.414755
[1930]	valid_0's binary_logloss: 0.405396	valid_1's binary_logloss: 0.414757
[1940]	valid_0's binary_logloss: 0.40535	valid_1's binary_logloss: 0.414756
[1950]	valid_0's binary_logloss: 0.4053	valid_1's binary_logloss: 0.414757
[1960]	valid_0's binary_logloss: 0.405248	valid_1's binary_logloss: 0.41476
[1970]	valid_0's binary_logloss: 0.405199	valid_1's binary_logloss: 0.414756
[1980]	valid_0's binary_logloss: 0.405147	valid_1's binary_logloss: 0.414753
[1990]	valid_0's binary_logloss: 0.405098	valid_1's binary_logloss: 0.414751
[2000]	valid_0's binary_logloss: 0.405045	valid_1's binary_logloss: 0.414753
[2010]	valid_0's binary_logloss: 0.404988	valid_1's binary_logloss: 0.414747
[2020]	valid_0's binary_logloss: 0.404939	valid_1's binary_logloss: 0.414753
[2030]	valid_0's binary_logloss: 0.404886	valid_1's binary_logloss: 0.414753
[20

...

[160]	valid_0's binary_logloss: 0.417772	valid_1's binary_logloss: 0.418234
[170]	valid_0's binary_logloss: 0.417561	valid_1's binary_logloss: 0.418058
[180]	valid_0's binary_logloss: 0.417352	valid_1's binary_logloss: 0.417883
[190]	valid_0's binary_logloss: 0.417168	valid_1's binary_logloss: 0.417738
[200]	valid_0's binary_logloss: 0.416994	valid_1's binary_logloss: 0.417595
[210]	valid_0's binary_logloss: 0.416834	valid_1's binary_logloss: 0.417465
[220]	valid_0's binary_logloss: 0.416681	valid_1's binary_logloss: 0.417346
[230]	valid_0's binary_logloss: 0.416522	valid_1's binary_logloss: 0.417231
[240]	valid_0's binary_logloss: 0.416383	valid_1's binary_logloss: 0.417127
[250]	valid_0's binary_logloss: 0.416256	valid_1's binary_logloss: 0.417049
[260]	valid_0's binary_logloss: 0.416132	valid_1's binary_logloss: 0.41696
[270]	valid_0's binary_logloss: 0.416009	valid_1's binary_logloss: 0.416886
[280]	valid_0's binary_logloss: 0.415883	valid_1's binary_logloss: 0.416808
[290]	valid_0

[1240]	valid_0's binary_logloss: 0.409074	valid_1's binary_logloss: 0.415053
[1250]	valid_0's binary_logloss: 0.409018	valid_1's binary_logloss: 0.415047
[1260]	valid_0's binary_logloss: 0.408961	valid_1's binary_logloss: 0.415044
[1270]	valid_0's binary_logloss: 0.408907	valid_1's binary_logloss: 0.41504
[1280]	valid_0's binary_logloss: 0.408848	valid_1's binary_logloss: 0.415035
[1290]	valid_0's binary_logloss: 0.40879	valid_1's binary_logloss: 0.415031
[1300]	valid_0's binary_logloss: 0.408735	valid_1's binary_logloss: 0.415025
[1310]	valid_0's binary_logloss: 0.408677	valid_1's binary_logloss: 0.415017
[1320]	valid_0's binary_logloss: 0.408622	valid_1's binary_logloss: 0.415017
[1330]	valid_0's binary_logloss: 0.40856	valid_1's binary_logloss: 0.415012
[1340]	valid_0's binary_logloss: 0.408506	valid_1's binary_logloss: 0.415008
[1350]	valid_0's binary_logloss: 0.408448	valid_1's binary_logloss: 0.415003
[1360]	valid_0's binary_logloss: 0.408394	valid_1's binary_logloss: 0.415001
[1

[2310]	valid_0's binary_logloss: 0.403475	valid_1's binary_logloss: 0.414784
[2320]	valid_0's binary_logloss: 0.403425	valid_1's binary_logloss: 0.414781
[2330]	valid_0's binary_logloss: 0.403376	valid_1's binary_logloss: 0.414782
[2340]	valid_0's binary_logloss: 0.403322	valid_1's binary_logloss: 0.41478
[2350]	valid_0's binary_logloss: 0.403274	valid_1's binary_logloss: 0.414781
[2360]	valid_0's binary_logloss: 0.403222	valid_1's binary_logloss: 0.414778
[2370]	valid_0's binary_logloss: 0.403176	valid_1's binary_logloss: 0.414782
[2380]	valid_0's binary_logloss: 0.403128	valid_1's binary_logloss: 0.414782
[2390]	valid_0's binary_logloss: 0.403083	valid_1's binary_logloss: 0.414782
[2400]	valid_0's binary_logloss: 0.403034	valid_1's binary_logloss: 0.414786
[2410]	valid_0's binary_logloss: 0.402991	valid_1's binary_logloss: 0.414784
[2420]	valid_0's binary_logloss: 0.402944	valid_1's binary_logloss: 0.414784
[2430]	valid_0's binary_logloss: 0.402892	valid_1's binary_logloss: 0.414787


[890]	valid_0's binary_logloss: 0.411105	valid_1's binary_logloss: 0.415504
[900]	valid_0's binary_logloss: 0.411043	valid_1's binary_logloss: 0.415497
[910]	valid_0's binary_logloss: 0.410975	valid_1's binary_logloss: 0.415488
[920]	valid_0's binary_logloss: 0.410919	valid_1's binary_logloss: 0.415488
[930]	valid_0's binary_logloss: 0.410858	valid_1's binary_logloss: 0.415483
[940]	valid_0's binary_logloss: 0.4108	valid_1's binary_logloss: 0.415478
[950]	valid_0's binary_logloss: 0.41074	valid_1's binary_logloss: 0.415472
[960]	valid_0's binary_logloss: 0.41068	valid_1's binary_logloss: 0.415467
[970]	valid_0's binary_logloss: 0.41062	valid_1's binary_logloss: 0.415456
[980]	valid_0's binary_logloss: 0.410559	valid_1's binary_logloss: 0.415452
[990]	valid_0's binary_logloss: 0.4105	valid_1's binary_logloss: 0.415446
[1000]	valid_0's binary_logloss: 0.410438	valid_1's binary_logloss: 0.415445
[1010]	valid_0's binary_logloss: 0.410376	valid_1's binary_logloss: 0.415435
[1020]	valid_0's 

[1960]	valid_0's binary_logloss: 0.405167	valid_1's binary_logloss: 0.415132
Early stopping, best iteration is:
[1868]	valid_0's binary_logloss: 0.405631	valid_1's binary_logloss: 0.415128
[0.4148858898168986, 0.4146883252626958, 0.41459811479927555, 0.4147769010588883, 0.41512814995148156]
cv: 0.4148154761778479
test mean: 0.19420425145654233


# Base Line     -- 0.415129

In [34]:
skf = StratifiedKFold(n_splits=5, random_state=1001, shuffle=True)
best_score = []
for index, (train_index, test_index) in enumerate(skf.split(train_csr, train_y)):
    lgb_model.fit(train_csr[train_index], train_y[train_index],
                  eval_set=[(train_csr[train_index], train_y[train_index]),
                            (train_csr[test_index], train_y[test_index])], early_stopping_rounds=100,verbose=10)
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    test_pred = lgb_model.predict_proba(predict_csr, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred

Training until validation scores don't improve for 100 rounds.
[10]	valid_0's binary_logloss: 0.458361	valid_1's binary_logloss: 0.458417
[20]	valid_0's binary_logloss: 0.44033	valid_1's binary_logloss: 0.440462
[30]	valid_0's binary_logloss: 0.43146	valid_1's binary_logloss: 0.431639
[40]	valid_0's binary_logloss: 0.426821	valid_1's binary_logloss: 0.427049
[50]	valid_0's binary_logloss: 0.424216	valid_1's binary_logloss: 0.424479
[60]	valid_0's binary_logloss: 0.422665	valid_1's binary_logloss: 0.422957
[70]	valid_0's binary_logloss: 0.421602	valid_1's binary_logloss: 0.42193
[80]	valid_0's binary_logloss: 0.420825	valid_1's binary_logloss: 0.421185
[90]	valid_0's binary_logloss: 0.420231	valid_1's binary_logloss: 0.420616
[100]	valid_0's binary_logloss: 0.41978	valid_1's binary_logloss: 0.420195
[110]	valid_0's binary_logloss: 0.419374	valid_1's binary_logloss: 0.419811
[120]	valid_0's binary_logloss: 0.419013	valid_1's binary_logloss: 0.419485
[130]	valid_0's binary_logloss: 0.4187

[1090]	valid_0's binary_logloss: 0.410874	valid_1's binary_logloss: 0.415402
[1100]	valid_0's binary_logloss: 0.410824	valid_1's binary_logloss: 0.415399
[1110]	valid_0's binary_logloss: 0.410773	valid_1's binary_logloss: 0.415385
[1120]	valid_0's binary_logloss: 0.410725	valid_1's binary_logloss: 0.415381
[1130]	valid_0's binary_logloss: 0.410673	valid_1's binary_logloss: 0.415373
[1140]	valid_0's binary_logloss: 0.410628	valid_1's binary_logloss: 0.41537
[1150]	valid_0's binary_logloss: 0.410582	valid_1's binary_logloss: 0.415366
[1160]	valid_0's binary_logloss: 0.410533	valid_1's binary_logloss: 0.415363
[1170]	valid_0's binary_logloss: 0.410488	valid_1's binary_logloss: 0.415363
[1180]	valid_0's binary_logloss: 0.410443	valid_1's binary_logloss: 0.415358
[1190]	valid_0's binary_logloss: 0.410393	valid_1's binary_logloss: 0.41535
[1200]	valid_0's binary_logloss: 0.410345	valid_1's binary_logloss: 0.415349
[1210]	valid_0's binary_logloss: 0.410298	valid_1's binary_logloss: 0.415346
[

KeyboardInterrupt: 